In [1]:
import numpy as np
import cancer

In [3]:
X, y = cancer.get_train()

In [4]:
def fit(X, y):
    N, D = X.shape
    cs = np.unique(y)
    C = len(cs)
    sigma2 = np.zeros(D)
    mus = []
    prior = []

    for c in cs:
        i = y == c
        X_c = X[i]
        prior.append(np.mean(i))
        mu_c = np.mean(X_c, axis = 0)
        mus.append(mu_c)
        sigma2 += np.sum((X_c - mu_c) ** 2, axis = 0)

    return cs, np.log(prior), mus, sigma2 / (1.0 * N - C)
        
def predict(model, X):
    N, D = X.shape
    cs, log_prior, mus, sigma2 = model
    p = []
    
    for i in range(len(cs)):
        Z = 0.5 * (X - mus[i]) ** 2 / sigma2
        p.append(log_prior[i] - np.sum([Z[:, j] for j in range(D)], axis = 0))
    return cs[np.argmax(p, axis = 0)]

(144, 16063)